In [1]:
import pandas as pd
df = pd.read_csv("../pre_train.csv")
df

,Unnamed: 0,/JS,/OpenAction,/AcroForm,/Annot,/Length,/Filter,/FlateDecode,/XObject,/CreationDate,...,eval,replace,fromCharCode,getAnnots,ip,xref_size,trailer_size,file_size,nof_stream,class
0,file_001,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.074150,0.875,1.0
1,file_002,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.033158,1.000,1.0
2,file_003,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.033736,1.000,1.0
3,file_011,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.037542,1.000,1.0
4,file_012,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.082498,0.875,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,file_5336,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.033395,1.000,1.0
2187,file_5337,0.000000,0.0,NaN,0.0,NaN,0.666667,0.333333,1.0,1.0,...,NaN,NaN,0.0,0.0,0.0,0.400000,0.500000,0.143468,0.375,0.0
2188,file_5341,0.666667,1.0,NaN,1.0,NaN,0.666667,0.666667,0.0,0.0,...,NaN,NaN,1.0,1.0,0.0,0.333333,0.333333,0.034405,1.000,1.0
2189,file_5342,0.000000,0.0,NaN,0.0,NaN,0.666667,0.666667,1.0,1.0,...,NaN,NaN,0.0,0.0,0.0,0.400000,0.500000,0.145757,0.500,0.0


#model import
- n_estimators : 학습 모델의 수, 많아질수록 성능 향상의 가능성이 있으나, 속도가 느려짐
- learning_rate : 학습률, 너무 크면 gradient 발산의 가능성이 있으며, 너무 작으면 학습이 느림
- max_depth : 최대 탐색 깊이, 너무 크면 과적합의 가능성, 너무 작으면 학습 성능 저하
- min_samples_split : 분할 종료 최소 샘플 수, 큰 수면 과적합을 막지만 학습 성능 저하 가능성
- min_samples_leaf : leaf node가 되기 위한 최소 샘플 수, min_samples_split과 비슷한 용도
- random_state : 특정 숫자로 고정 시에 실행 시마다 고정된 결과 반환

In [3]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=500, learning_rate=0.2, max_depth=4, random_state = 32)

In [8]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]



In [9]:
print(X)
print(y)

     Unnamed: 0       /JS  /OpenAction  /AcroForm  /Annot  /Length   /Filter  \
0      file_001  0.666667          1.0        NaN     1.0      NaN  0.666667   
1      file_002  0.666667          1.0        NaN     1.0      NaN  0.666667   
2      file_003  0.666667          1.0        NaN     1.0      NaN  0.666667   
3      file_011  0.666667          1.0        NaN     1.0      NaN  0.666667   
4      file_012  0.666667          1.0        NaN     1.0      NaN  0.666667   
...         ...       ...          ...        ...     ...      ...       ...   
2186  file_5336  0.666667          1.0        NaN     1.0      NaN  0.666667   
2187  file_5337  0.000000          0.0        NaN     0.0      NaN  0.666667   
2188  file_5341  0.666667          1.0        NaN     1.0      NaN  0.666667   
2189  file_5342  0.000000          0.0        NaN     0.0      NaN  0.666667   
2190  file_5343  0.666667          1.0        NaN     1.0      NaN  0.666667   

      /FlateDecode  /XObject  /Creation

In [ ]:
# 모델 학습
model.fit(X, y)

In [6]:
from sklearn.metrics import accuracy_score
y = model.predict(X)

NotFittedError: need to call fit or load_model beforehand